# Config

In [1]:
import pandas as pd

In [2]:
def evaluate_sessions(_events_df, _participant_id):
  sub_df = _events_df[_events_df['participant'] == _participant_id]
  return sub_df[sub_df['event'] == 'survey | load'].shape[0]

In [3]:
def evaluate_reading_time(_events_df, _participant_id, _d):
  sub_df = _events_df[_events_df['participant'] == _participant_id]
  sub_df = sub_df.sort_values('t')

  events_list = sub_df.to_dict(orient='records')

  visibles = [e['t'] for e in events_list if e['event'] == f'{_d} | visible | text']
  if len(visibles) == 0:
    return 0

  invisibles = [e['t'] for e in events_list if e['event'] == f'{_d} | invisible | text' and e['t'] > visibles[0]]
  visibility_spans = [(visibles[i], invisibles[i]) for i in range(min(len(visibles), len(invisibles)))]
  return sum([e[1] - e[0] for e in visibility_spans]) / 1e3


def evaluate_reading_time_d1(_events_df, _participant_id):
  return evaluate_reading_time(_events_df, _participant_id, 'd1')


def evaluate_reading_time_d2(_events_df, _participant_id):
  return evaluate_reading_time(_events_df, _participant_id, 'd2')

# Load data

In [4]:
events_df = pd.read_csv(f'./raw_data/survey_event.csv', index_col=0, names=['id', 'event', 't', 'participant'], header=0)
events_df.head()

,event,t,participant
id,,,
1,d1 | visible | text,1719827039609,UEFSVEVDSVBBTlRFXzgy
2,d1 | invisible | q6,1719827039597,UEFSVEVDSVBBTlRFXzgy
3,d1 | invisible | q5,1719827039607,UEFSVEVDSVBBTlRFXzgy
4,survey | load,1719827039268,UEFSVEVDSVBBTlRFXzgy
5,d1 | invisible | q2,1719827039605,UEFSVEVDSVBBTlRFXzgy


In [5]:
participant_df = pd.read_csv(f'./raw_data/participant.csv')
participant_df.head()

,id,creation_date,participant_group,participant_type,submission_date,answers_d1,answers_d2,demographic_info
0,1,2024-07-03 09:47:27.324685,A,ADMINISTRATIVE,2024-07-03 10:04:27.799306,530cf9cf-599a-4475-bb7c-845f0b1ee745,59bea0a7-e80a-4912-8f74-206ed6e6da88,03b757a4-1d34-4402-a7f6-076c3e3df170
1,10,2024-07-03 09:48:26.727449,B,ADMINISTRATIVE,2024-07-03 10:04:52.056617,8e638297-f55f-44cd-861d-8232d75a1e42,4f47663a-17b7-47d8-ba57-4d201b59c0f3,4c7743a4-24f8-4ade-b013-2d0e8c45ba8f
2,11,2024-07-03 09:48:24.888290,A,ADMINISTRATIVE,2024-07-03 10:03:03.416511,d53c8d4f-d8d3-40f3-a542-9f28368bbe4c,2d8a2714-9646-42d9-a5ec-95145019e3d7,62dc7caa-b574-4c9f-90c2-b4e42c0c3e0f
3,13,2024-07-03 09:48:18.365622,C,ADMINISTRATIVE,2024-07-03 10:05:14.581160,bc2cece9-fdf8-4e94-9b3b-9d85202dcf17,76bc07e9-1c23-4830-ae9f-37d4498f6a66,c84fffc1-00e0-4727-b240-95733855664c
4,14,2024-07-03 09:48:09.720491,B,ADMINISTRATIVE,2024-07-03 10:07:08.696679,aa7baabe-eb18-4267-898c-a04dc84f9303,c7b1fb8b-c014-4bbd-b3da-babbb3960da3,d9a0dba9-0446-444f-82dc-30c6813fd6d1


In [6]:
demographic_df = pd.read_csv(f'./raw_data/demographic_info.csv')
demographic_df.head()

,uuid,age_range,education_fields,education_level,employment,employment_fields,italian_knowledge
0,01a08668-7ff0-407c-9ca7-b3199a7e67e1,45-54,JURIDICAL,MASTER,EMPLOYEE,ADMINISTRATIVE,M
1,02f20641-0047-4e69-86c4-be3a9d07e841,35-44,JURIDICAL,MASTER,EMPLOYEE,ADMINISTRATIVE,M
2,03b757a4-1d34-4402-a7f6-076c3e3df170,25-34,JURIDICAL,BACHELOR,EMPLOYEE,ADMINISTRATIVE,M
3,052070fa-c886-48c1-8bcc-33a1f91bf703,25-34,HUMANISTIC,BACHELOR,UNEMPLOYED,EDUCATION,C2
4,08aeabdc-5ee0-4169-931f-78c2e10a2f09,18-24,HUMANISTIC | JURIDICAL,BACHELOR,STUDENT,EDUCATION | SALES | ARTISTIC,B1


In [7]:
answers_d1_df = pd.read_csv(f'./raw_data/answers_d1.csv')
answers_d1_df.head()

,uuid,q1,q2,q3,q4,q5,q6,rate
0,01bb1c25-d8f1-4e72-8122-7494e20cb63d,1.0,3.0,1.0,1.0,1.0,1.0,5.0
1,0460198c-38b8-4f9e-bc18-3248ec02b684,1.0,5.0,1.0,1.0,1.0,1.0,5.0
2,05a20d99-56c4-4956-9658-f657bb2450ac,1.0,2.0,1.0,2.0,5.0,1.0,3.0
3,05cb4ae0-8e25-410e-803b-0d0f0ec74ad8,NaN,1.0,2.0,1.0,1.0,1.0,3.0
4,061c2890-fb36-4882-88a8-57cedd8eb4ea,1.0,1.0,1.0,1.0,5.0,3.0,4.0


In [8]:
answers_d2_df = pd.read_csv(f'./raw_data/answers_d2.csv')
answers_d2_df.head()

,uuid,q1,q2,q3,q4,q5,q6,rate
0,059b2ca0-6075-4083-8664-1b359baab5e2,1.0,1.0,1.0,2.0,1.0,1.0,5.0
1,0609ea66-6a92-496e-9570-33f8831ba972,1.0,1.0,1.0,2.0,1.0,1.0,4.0
2,06dac9a0-277a-4a59-aafa-e23daae8fca8,2.0,1.0,3.0,1.0,1.0,1.0,5.0
3,06df80d5-ba51-49a9-8f8a-f97faa365c21,1.0,2.0,4.0,2.0,1.0,1.0,3.0
4,0a6145fe-4e9f-4013-ad95-81e6d8be7ac1,5.0,5.0,5.0,5.0,5.0,5.0,4.0


# Merge

In [9]:
merged_df = participant_df.merge(demographic_df, left_on='demographic_info', right_on='uuid', how='left')
merged_df = merged_df.merge(answers_d1_df.add_prefix('d1_'), left_on='answers_d1', right_on='d1_uuid', how='left')
merged_df = merged_df.merge(answers_d2_df.add_prefix('d2_'), left_on='answers_d2', right_on='d2_uuid', how='left')
merged_df = merged_df.drop(columns=['uuid', 'demographic_info', 'answers_d1', 'd1_uuid', 'answers_d2', 'd2_uuid'])
merged_df.head()

,id,creation_date,participant_group,participant_type,submission_date,age_range,education_fields,education_level,employment,employment_fields,...,d1_q5,d1_q6,d1_rate,d2_q1,d2_q2,d2_q3,d2_q4,d2_q5,d2_q6,d2_rate
0,1,2024-07-03 09:47:27.324685,A,ADMINISTRATIVE,2024-07-03 10:04:27.799306,25-34,JURIDICAL,BACHELOR,EMPLOYEE,ADMINISTRATIVE,...,1.0,1.0,4.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0
1,10,2024-07-03 09:48:26.727449,B,ADMINISTRATIVE,2024-07-03 10:04:52.056617,35-44,OTHER,HIGH,EMPLOYEE,IT | OTHER,...,1.0,1.0,3.0,1.0,4.0,2.0,1.0,1.0,2.0,3.0
2,11,2024-07-03 09:48:24.888290,A,ADMINISTRATIVE,2024-07-03 10:03:03.416511,45-54,JURIDICAL,MASTER,EMPLOYEE,ADMINISTRATIVE,...,2.0,3.0,3.0,1.0,1.0,1.0,1.0,1.0,2.0,3.0
3,13,2024-07-03 09:48:18.365622,C,ADMINISTRATIVE,2024-07-03 10:05:14.581160,45-54,OTHER,HIGH,EMPLOYEE,ADMINISTRATIVE,...,2.0,1.0,3.0,1.0,1.0,1.0,2.0,1.0,1.0,3.0
4,14,2024-07-03 09:48:09.720491,B,ADMINISTRATIVE,2024-07-03 10:07:08.696679,55-64,JURIDICAL,MASTER,EMPLOYEE,ADMINISTRATIVE,...,1.0,1.0,2.0,1.0,2.0,2.0,2.0,1.0,1.0,3.0


# Eval features

In [10]:
merged_df['duration'] = (pd.to_datetime(merged_df['submission_date']) - pd.to_datetime(merged_df['creation_date'])).view('int64') // 10**9
merged_df['sessions'] = merged_df['id'].map(lambda x: evaluate_sessions(events_df, x))
merged_df['d1_reading_time'] = merged_df['id'].map(lambda x: evaluate_reading_time_d1(events_df, x))
merged_df['d2_reading_time'] = merged_df['id'].map(lambda x: evaluate_reading_time_d2(events_df, x))

C:\Users\marco\AppData\Local\Temp\ipykernel_36472\1739656136.py:1: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  merged_df['duration'] = (pd.to_datetime(merged_df['submission_date']) - pd.to_datetime(merged_df['creation_date'])).view('int64') // 10**9


In [11]:
merged_df['d1_correctness'] = (merged_df[['d1_q1', 'd1_q2', 'd1_q3', 'd1_q4', 'd1_q5', 'd1_q6']] == 1).sum(axis=1)
merged_df['d2_correctness'] = (merged_df[['d2_q1', 'd2_q2', 'd2_q3', 'd2_q4', 'd2_q5', 'd2_q6']] == 1).sum(axis=1)

merged_df['d1_insecurity'] = (merged_df[['d1_q1', 'd1_q2', 'd1_q3', 'd1_q4', 'd1_q5', 'd1_q6']] == 5).sum(axis=1) > 0
merged_df['d2_insecurity'] = (merged_df[['d2_q1', 'd2_q2', 'd2_q3', 'd2_q4', 'd2_q5', 'd2_q6']] == 5).sum(axis=1) > 0

merged_df['d1_insecurity_count'] = (merged_df[['d1_q1', 'd1_q2', 'd1_q3', 'd1_q4', 'd1_q5', 'd1_q6']] == 5).sum(axis=1)
merged_df['d2_insecurity_count'] = (merged_df[['d2_q1', 'd2_q2', 'd2_q3', 'd2_q4', 'd2_q5', 'd2_q6']] == 5).sum(axis=1)

In [12]:
merged_df

,id,creation_date,participant_group,participant_type,submission_date,age_range,education_fields,education_level,employment,employment_fields,...,duration,sessions,d1_reading_time,d2_reading_time,d1_correctness,d2_correctness,d1_insecurity,d2_insecurity,d1_insecurity_count,d2_insecurity_count
0,1,2024-07-03 09:47:27.324685,A,ADMINISTRATIVE,2024-07-03 10:04:27.799306,25-34,JURIDICAL,BACHELOR,EMPLOYEE,ADMINISTRATIVE,...,1020,1,499.972,226.652,5,5,False,False,0,0
1,10,2024-07-03 09:48:26.727449,B,ADMINISTRATIVE,2024-07-03 10:04:52.056617,35-44,OTHER,HIGH,EMPLOYEE,IT | OTHER,...,985,1,383.870,233.470,4,3,False,False,0,0
2,11,2024-07-03 09:48:24.888290,A,ADMINISTRATIVE,2024-07-03 10:03:03.416511,45-54,JURIDICAL,MASTER,EMPLOYEE,ADMINISTRATIVE,...,878,1,309.125,221.966,2,5,False,False,0,0
3,13,2024-07-03 09:48:18.365622,C,ADMINISTRATIVE,2024-07-03 10:05:14.581160,45-54,OTHER,HIGH,EMPLOYEE,ADMINISTRATIVE,...,1016,1,416.598,316.136,4,5,False,False,0,0
4,14,2024-07-03 09:48:09.720491,B,ADMINISTRATIVE,2024-07-03 10:07:08.696679,55-64,JURIDICAL,MASTER,EMPLOYEE,ADMINISTRATIVE,...,1138,1,323.449,289.004,4,3,False,False,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,UEFSVEVDSVBBTlRFXzU1,2024-07-04 13:21:46.597038,C,FOREIGNER,2024-07-04 13:33:35.799668,35-44,OTHER,HIGH,EMPLOYEE,SALES,...,709,1,22.162,74.442,1,2,True,True,3,1
165,UEFSVEVDSVBBTlRFXzUw,2024-07-01 13:08:59.918366,A,FOREIGNER,2024-07-01 13:30:02.402533,45-54,HUMANISTIC,BACHELOR,EMPLOYEE,OTHER,...,1262,1,620.984,242.522,4,5,False,False,0,0
166,UEFSVEVDSVBBTlRFXzUx,2024-07-01 13:13:25.434286,B,FOREIGNER,2024-07-01 13:54:49.149476,25-34,HUMANISTIC,BACHELOR,UNEMPLOYED,EDUCATION,...,2483,2,1157.067,612.700,4,5,False,False,0,0
167,UEFSVEVDSVBBTlRFXzUy,2024-07-03 15:18:35.753497,A,FOREIGNER,NaN,35-44,OTHER,BACHELOR,EMPLOYEE,OTHER,...,-9223372037,4,41.841,0.000,0,0,False,False,0,0


# Basic filtering

In [13]:
filtered_merged_df = merged_df.copy()
merged_df['exclusion_reason'] = ''

print(filtered_merged_df.shape[0])
filtered_merged_df['participant_type'].value_counts()

169


participant_type
FOREIGNER         83
REGULAR           70
ADMINISTRATIVE    16
Name: count, dtype: int64

In [14]:
# Remove "NOT TERMINATED"
ids = filtered_merged_df[filtered_merged_df['submission_date'].isna()]['id'].tolist()
print(ids)
filtered_merged_df = filtered_merged_df[~filtered_merged_df['id'].isin(ids)]
merged_df.loc[merged_df['id'].isin(ids), ['exclusion_reason']] = 'NOT TERMINATED'

print(filtered_merged_df.shape[0])
filtered_merged_df['participant_type'].value_counts()

['S1', 'S12', 'S14', 'S31', 'S46', 'S72', 'UEFSVEVDSVBBTlRFXzE1MA==', 'UEFSVEVDSVBBTlRFXzE1OQ==', 'UEFSVEVDSVBBTlRFXzE2MQ==', 'UEFSVEVDSVBBTlRFXzEzOA==', 'UEFSVEVDSVBBTlRFXzg4', 'UEFSVEVDSVBBTlRFXzUy']
157


participant_type
FOREIGNER         76
REGULAR           65
ADMINISTRATIVE    16
Name: count, dtype: int64

In [15]:
# Remove "DATA ERROR"
ids = filtered_merged_df[filtered_merged_df['d1_q1'].isna() | filtered_merged_df['d2_q1'].isna() |
                         filtered_merged_df['d1_q2'].isna() | filtered_merged_df['d2_q2'].isna() |
                         filtered_merged_df['d1_q3'].isna() | filtered_merged_df['d2_q3'].isna() |
                         filtered_merged_df['d1_q4'].isna() | filtered_merged_df['d2_q4'].isna() |
                         filtered_merged_df['d1_q5'].isna() | filtered_merged_df['d2_q5'].isna() |
                         filtered_merged_df['d1_q6'].isna() | filtered_merged_df['d2_q6'].isna()]['id'].tolist()
print(ids)
filtered_merged_df = filtered_merged_df[~filtered_merged_df['id'].isin(ids)]
merged_df.loc[merged_df['id'].isin(ids), ['exclusion_reason']] = 'DATA ERROR'

print(filtered_merged_df.shape[0])
filtered_merged_df['participant_type'].value_counts()

['S11', 'S52', 'S54', 'S62', 'S63', 'S68', 'S7', 'S8', 'UEFSVEVDSVBBTlRFXzE0Mw==', 'UEFSVEVDSVBBTlRFXzE1NQ==', 'UEFSVEVDSVBBTlRFXzE2Ng==', 'UEFSVEVDSVBBTlRFXzE3Mw==']
145


participant_type
FOREIGNER         68
REGULAR           61
ADMINISTRATIVE    16
Name: count, dtype: int64

In [16]:
# Remove "LOW TIME"
ids = filtered_merged_df[filtered_merged_df['duration'] < 10 * 60]['id'].tolist()
print(ids)
filtered_merged_df = filtered_merged_df[~filtered_merged_df['id'].isin(ids)]
merged_df.loc[merged_df['id'].isin(ids), ['exclusion_reason']] = 'LOW TIME'

print(filtered_merged_df.shape[0])
filtered_merged_df['participant_type'].value_counts()

['S10', 'S44', 'S45', 'S47', 'S5', 'S50', 'S6', 'S61', 'S71', 'UEFSVEVDSVBBTlRFXzE0MQ==', 'UEFSVEVDSVBBTlRFXzE3Mg==', 'UEFSVEVDSVBBTlRFXzEwNQ==', 'UEFSVEVDSVBBTlRFXzEyNQ==', 'UEFSVEVDSVBBTlRFXzg=', 'UEFSVEVDSVBBTlRFXzgz', 'UEFSVEVDSVBBTlRFXzkw']
129


participant_type
FOREIGNER         59
REGULAR           55
ADMINISTRATIVE    15
Name: count, dtype: int64

In [17]:
# Remove "BOTH CONTROL QUESTION"
ids = filtered_merged_df[(filtered_merged_df['d1_q6'] != 1) & (filtered_merged_df['d2_q5'] != 1)]['id'].tolist()
print(ids)
filtered_merged_df = filtered_merged_df[~filtered_merged_df['id'].isin(ids)]
merged_df.loc[merged_df['id'].isin(ids), ['exclusion_reason']] = 'BOTH CONTROL QUESTION'

print(filtered_merged_df.shape[0])
filtered_merged_df['participant_type'].value_counts()

['S16', 'S19', 'S26', 'S30', 'S37', 'S38', 'S40', 'S43', 'S70', 'UEFSVEVDSVBBTlRFXzE0Nw==', 'UEFSVEVDSVBBTlRFXzE0OQ==', 'UEFSVEVDSVBBTlRFXzEzMQ==', 'UEFSVEVDSVBBTlRFXzg0', 'UEFSVEVDSVBBTlRFXzU1']
115


participant_type
REGULAR           51
FOREIGNER         49
ADMINISTRATIVE    15
Name: count, dtype: int64

In [18]:
# Remove "NEGATIVE READING TIME"
ids = filtered_merged_df[(filtered_merged_df['d1_reading_time'] <= 0) | (filtered_merged_df['d2_reading_time'] <= 0)]['id'].tolist()
print(ids)
filtered_merged_df = filtered_merged_df[~filtered_merged_df['id'].isin(ids)]
merged_df.loc[merged_df['id'].isin(ids), ['exclusion_reason']] = 'NEGATIVE READING TIME'

print(filtered_merged_df.shape[0])
filtered_merged_df['participant_type'].value_counts()

['S2', 'S65', 'UEFSVEVDSVBBTlRFXzE0Mg==', 'UEFSVEVDSVBBTlRFXzEwNw==', 'UEFSVEVDSVBBTlRFXzExMA==', 'UEFSVEVDSVBBTlRFXzEzNw==']
109


participant_type
FOREIGNER         47
REGULAR           47
ADMINISTRATIVE    15
Name: count, dtype: int64

In [19]:
# Remove "LOW READING TIME"
ids = filtered_merged_df[(filtered_merged_df['d1_reading_time'] <= 60) | (filtered_merged_df['d2_reading_time'] <= 60)]['id'].tolist()
print(ids)
filtered_merged_df = filtered_merged_df[~filtered_merged_df['id'].isin(ids)]
merged_df.loc[merged_df['id'].isin(ids), ['exclusion_reason']] = 'LOW READING TIME'

print(filtered_merged_df.shape[0])
filtered_merged_df['participant_type'].value_counts()

['S22', 'S23', 'S29', 'S41', 'S48', 'S51', 'S55', 'S56', 'S57', 'S58', 'S60', 'S66', 'S67', 'UEFSVEVDSVBBTlRFXzE0MA==']
95


participant_type
REGULAR           46
FOREIGNER         34
ADMINISTRATIVE    15
Name: count, dtype: int64

In [20]:
# Remove "MANUALLY"
ids = [
    'S9',
    'S32',
    'S34',
    'S64',
    'S69',
    'UEFSVEVDSVBBTlRFXzEzNQ==',
    'UEFSVEVDSVBBTlRFXzEwNA==',
    'UEFSVEVDSVBBTlRFXzE3Ng==',
    'UEFSVEVDSVBBTlRFXzEyNg==',
    'UEFSVEVDSVBBTlRFXzEzNg=='
]
print(ids)
filtered_merged_df = filtered_merged_df[~filtered_merged_df['id'].isin(ids)]
merged_df.loc[merged_df['id'].isin(ids), ['exclusion_reason']] = 'MANUALLY'

print(filtered_merged_df.shape[0])
filtered_merged_df['participant_type'].value_counts()

['S9', 'S32', 'S34', 'S64', 'S69', 'UEFSVEVDSVBBTlRFXzEzNQ==', 'UEFSVEVDSVBBTlRFXzEwNA==', 'UEFSVEVDSVBBTlRFXzE3Ng==', 'UEFSVEVDSVBBTlRFXzEyNg==', 'UEFSVEVDSVBBTlRFXzEzNg==']
85


participant_type
REGULAR           41
FOREIGNER         29
ADMINISTRATIVE    15
Name: count, dtype: int64

# Save

In [21]:
COLUMNS = [
    'id',
    'participant_type',
    'participant_group',
    'duration',
    'sessions',
    # D1
    'd1_reading_time',
    'd1_q1',
    'd1_q2',
    'd1_q3',
    'd1_q4',
    'd1_q5',
    'd1_q6',
    'd1_rate',
    'd1_correctness',
    'd1_insecurity',
    'd1_insecurity_count',
    # D2
    'd2_reading_time',
    'd2_q1',
    'd2_q2',
    'd2_q3',
    'd2_q4',
    'd2_q5',
    'd2_q6',
    'd2_rate',
    'd2_correctness',
    'd2_insecurity',
    'd2_insecurity_count',
    # Demographics
    'age_range',
    'education_fields',
    'education_level',
    'employment',
    'employment_fields',
    'italian_knowledge'
]
COLUMNS_2 = COLUMNS + ['exclusion_reason']

In [22]:
merged_df[COLUMNS_2].to_csv(f'./final_data/participants.csv', index=None)
merged_df[COLUMNS_2].to_excel(f'./final_data/participants.xlsx', index=None)
filtered_merged_df[COLUMNS].to_csv(f'./final_data/filtered_participants.csv', index=None)
filtered_merged_df[COLUMNS].to_excel(f'./final_data/filtered_participants.xlsx', index=None)